In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('dkk20-21data.txt')
winter2 = pd.read_csv ('dkk21-22data.txt')
winter3 = pd.read_csv ('dkk22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
180,DKK,2021-12-06 22:53:00,T,-SN
181,DKK,2021-12-06 23:10:00,T,-SN
182,DKK,2021-12-06 23:19:00,T,-SN
183,DKK,2021-12-06 23:53:00,T,-SN
184,DKK,2021-12-07 00:53:00,0.25,-SN
...,...,...,...,...
3171,DKK,2022-02-27 21:15:00,T,-SN
3172,DKK,2022-02-27 21:25:00,T,-SN BR
3173,DKK,2022-02-27 21:29:00,T,-SN
3174,DKK,2022-02-27 21:40:00,T,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
dkk = LocationInfo(
    'Dunkirk', 'New York', 
    latitude= 42.4795, 
    longitude= -79.3339, 
    timezone='America/New_York'
)

dkk

LocationInfo(name='Dunkirk', region='New York', timezone='America/New_York', latitude=42.4795, longitude=-79.3339)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(dkk.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(dkk.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(dkk.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:28:04  2020-12-01 16:44:55  2020-12-01
1   2020-12-02 07:29:07  2020-12-02 16:44:39  2020-12-02
2   2020-12-03 07:30:09  2020-12-03 16:44:26  2020-12-03
3   2020-12-04 07:31:09  2020-12-04 16:44:15  2020-12-04
4   2020-12-05 07:32:08  2020-12-05 16:44:07  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 07:00:31  2021-02-24 18:01:02  2021-02-24
86  2021-02-25 06:58:57  2021-02-25 18:02:17  2021-02-25
87  2021-02-26 06:57:22  2021-02-26 18:03:31  2021-02-26
88  2021-02-27 06:55:47  2021-02-27 18:04:45  2021-02-27
89  2021-02-28 06:54:11  2021-02-28 18:05:59  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:27:48  2021-12-01 16:44:59  2021-12-01
1   2021-12-02 07:28:52  2021-12-02 16:44:43  2021-12-02
2   2021-12-03 07:29:54  2021-12-03 16:44:29  2021-12-03
3   2021-12-04 07:30:55  2021-12-04 16:44:18  2021-12-04
4   2021-12-05 07:31:54  2021-12-05 16:44:09  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 07:00:53  2022-02-24 18:00:44  2022-02-24
86  2022-02-25 06:59:20  2022-02-25 18:01:59  2022-02-25
87  2022-02-26 06:57:45  2022-02-26 18:03:13  2022-02-26
88  2022-02-27 06:56:10  2022-02-27 18:04:27  2022-02-27
89  2022-02-28 06:54:34  2022-02-28 18:05:41  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:27:33  2022-12-01 16:45:03  2022-12-01
1   2022-12-02 07:28:36  2022-12-02 16:44:46  2022-12-02
2   2022-12-03 07:29:39  2022-12-03 16:44:32  2022-12-03
3   2022-12-04 07:30:40  2022-12-04 16:44:20  2022-12-04
4   2022-12-05 07:31:40  2022-12-05 16:44:10  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 07:01:15  2023-02-24 18:00:26  2023-02-24
86  2023-02-25 06:59:42  2023-02-25 18:01:41  2023-02-25
87  2023-02-26 06:58:08  2023-02-26 18:02:55  2023-02-26
88  2023-02-27 06:56:33  2023-02-27 18:04:09  2023-02-27
89  2023-02-28 06:54:58  2023-02-28 18:05:23  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DKK,2020-12-01 07:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
1,DKK,2020-12-01 08:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
2,DKK,2020-12-01 10:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
3,DKK,2020-12-01 11:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
4,DKK,2020-12-01 12:53:00,0.0001,SN FG,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
...,...,...,...,...,...,...,...
324,DKK,2021-02-23 12:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47
325,DKK,2021-02-23 13:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47
326,DKK,2021-02-23 14:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47
327,DKK,2021-02-23 15:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DKK,2021-12-06 22:53:00,0.0001,-SN,2021-12-06,2021-12-06 07:32:52,2021-12-06 16:44:02
1,DKK,2021-12-06 23:53:00,0.0001,-SN,2021-12-06,2021-12-06 07:32:52,2021-12-06 16:44:02
2,DKK,2021-12-07 00:53:00,0.2500,-SN,2021-12-07,2021-12-07 07:33:49,2021-12-07 16:43:58
3,DKK,2021-12-07 02:53:00,0.0001,-SN,2021-12-07,2021-12-07 07:33:49,2021-12-07 16:43:58
4,DKK,2021-12-07 03:53:00,0.0001,-SN,2021-12-07,2021-12-07 07:33:49,2021-12-07 16:43:58
...,...,...,...,...,...,...,...
367,DKK,2022-02-26 22:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:57:45,2022-02-26 18:03:13
368,DKK,2022-02-26 23:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:57:45,2022-02-26 18:03:13
369,DKK,2022-02-27 18:53:00,0.0001,-SN,2022-02-27,2022-02-27 06:56:10,2022-02-27 18:04:27
370,DKK,2022-02-27 19:53:00,0.0001,-SN,2022-02-27,2022-02-27 06:56:10,2022-02-27 18:04:27


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DKK,2022-12-01 00:53:00,0.0001,-SN,2022-12-01,2022-12-01 07:27:33,2022-12-01 16:45:03
1,DKK,2022-12-01 01:53:00,1.0200,-SN,2022-12-01,2022-12-01 07:27:33,2022-12-01 16:45:03
2,DKK,2022-12-01 03:53:00,0.0001,-SN,2022-12-01,2022-12-01 07:27:33,2022-12-01 16:45:03
3,DKK,2022-12-01 04:53:00,0.0001,-SN,2022-12-01,2022-12-01 07:27:33,2022-12-01 16:45:03
4,DKK,2022-12-01 05:53:00,0.2500,-SN,2022-12-01,2022-12-01 07:27:33,2022-12-01 16:45:03
...,...,...,...,...,...,...,...
262,DKK,2023-02-24 17:53:00,0.0001,-SN,2023-02-24,2023-02-24 07:01:15,2023-02-24 18:00:26
263,DKK,2023-02-24 18:53:00,0.0001,-SN,2023-02-24,2023-02-24 07:01:15,2023-02-24 18:00:26
264,DKK,2023-02-24 19:53:00,0.0001,-SN,2023-02-24,2023-02-24 07:01:15,2023-02-24 18:00:26
265,DKK,2023-02-25 19:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:59:42,2023-02-25 18:01:41


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DKK,2020-12-01 07:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
1,DKK,2020-12-01 08:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
2,DKK,2020-12-01 10:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
3,DKK,2020-12-01 11:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
4,DKK,2020-12-01 12:53:00,0.0001,SN FG,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
...,...,...,...,...,...,...,...
323,DKK,2021-02-23 11:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47
324,DKK,2021-02-23 12:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47
325,DKK,2021-02-23 13:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47
326,DKK,2021-02-23 14:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
1,DKK,2020-12-01 08:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55,2020-12-01 08:28:04,2020-12-01 15:44:55
2,DKK,2020-12-01 10:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55,2020-12-01 08:28:04,2020-12-01 15:44:55
3,DKK,2020-12-01 11:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55,2020-12-01 08:28:04,2020-12-01 15:44:55
4,DKK,2020-12-01 12:53:00,0.0001,SN FG,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55,2020-12-01 08:28:04,2020-12-01 15:44:55
5,DKK,2020-12-01 13:53:00,0.0001,SN FG,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55,2020-12-01 08:28:04,2020-12-01 15:44:55
...,...,...,...,...,...,...,...,...,...
323,DKK,2021-02-23 11:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47,2021-02-23 08:02:03,2021-02-23 16:59:47
324,DKK,2021-02-23 12:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47,2021-02-23 08:02:03,2021-02-23 16:59:47
325,DKK,2021-02-23 13:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47,2021-02-23 08:02:03,2021-02-23 16:59:47
326,DKK,2021-02-23 14:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:02:03,2021-02-23 17:59:47,2021-02-23 08:02:03,2021-02-23 16:59:47


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.11651111111111116
2021-2022: 0.07255974025974025
2022-2023: 0.133379381443299


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
7,DKK,2020-12-01 16:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
8,DKK,2020-12-01 17:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
9,DKK,2020-12-01 18:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
10,DKK,2020-12-01 19:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
11,DKK,2020-12-01 20:53:00,0.2500,-SN,2020-12-01,2020-12-01 07:28:04,2020-12-01 16:44:55
...,...,...,...,...,...,...,...
312,DKK,2021-02-21 05:53:00,0.0001,-SN,2021-02-21,2021-02-21 07:05:06,2021-02-21 17:57:17
319,DKK,2021-02-22 19:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:35,2021-02-22 17:58:32
320,DKK,2021-02-22 21:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:35,2021-02-22 17:58:32
321,DKK,2021-02-22 22:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:35,2021-02-22 17:58:32


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31288\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.07184054054054054
2021-2022: 0.03171875
2022-2023: 0.08024999999999999


In [33]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('SO.dkk.csv', index=False)